In [8]:
# Importing needed libraries:
# Make this a function named acquire.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.linear_model import TweedieRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import os
import env

In [9]:
import acquire
import luke_wrangle
import prepare

acquire.py functions loaded successfully


ModuleNotFoundError: No module named 'luke_wrangle'

In [ ]:
df = acquire.get_zillow_data()
df.head()

In [ ]:
pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

## Data Preparation

In [ ]:
# Finding out how many of the columns have at least half of their rows as nulls:

df.isna().sum() > (df.shape[0]/2 + 1000)

In [ ]:
df = df.drop(columns = {'airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft',
                        'taxdelinquencyyear', 'buildingclasstypeid', 'decktypeid', 'finishedfloor1squarefeet',
                        'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
                        'finishedsquarefeet6', 'fireplacecnt', 'hashottuborspa', 'poolsizesum',
                        'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'storytypeid', 'threequarterbathnbr',
                        'typeconstructiontypeid', 'yardbuildingsqft17', 'yardbuildingsqft26', 'numberofstories',
                        'fireplaceflag', 'taxdelinquencyflag', 'buildingqualitytypeid', 'garagecarcnt', 'garagetotalsqft', 'poolcnt', 'regionidneighborhood', 'propertyzoningdesc' })

In [ ]:
df.head()

In [ ]:
print(df.info())
print(df.shape)

In [ ]:
df.bathroomcnt.hist()

In [ ]:
df.bedroomcnt.hist()

In [ ]:
# Creating a diaginal heatmap that allows a clear view of the correlated features (independent variables)

In [ ]:
corr = df.corr()
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(500, 20, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df.info()

In [ ]:
print(df.bathroomcnt.isna().sum())
print(df.bedroomcnt.isna().sum())
print(df.calculatedfinishedsquarefeet.isna().sum())

In [ ]:
# For the MVP, these are the 3 features (columns) to focus on. 

In [ ]:
df.calculatedfinishedsquarefeet.hist()

In [ ]:
# Let's go with the mean:

df.calculatedfinishedsquarefeet.mean()

In [ ]:
df['calculatedfinishedsquarefeet'] = df['calculatedfinishedsquarefeet'].fillna((df['calculatedfinishedsquarefeet'].mean()))

In [10]:
df.head()

NameError: name 'df' is not defined

## Using Corrplot

Attempting to use the corrplot function from biokit.viz

In [11]:
df_corr = df[['parcelid', 'bathroomcnt', 'bedroomcnt', '']]
df_corr.head()

NameError: name 'df' is not defined

In [12]:
# Isolating the object columns so I can get a realistic heatmap:

# Creating the mask and if a column is an int/float/object
mask = np.array(df.dtypes == "object")

# Creating the boolean mask:
object_cols = df.iloc[:, mask].columns.tolist()
object_cols

NameError: name 'df' is not defined

In [13]:
numeric_type_cols = df.iloc[:, ~mask].columns.tolist()
numeric_type_cols

NameError: name 'df' is not defined

In [14]:
df_corr = df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']]
# df_corr.drop(columns = {'id', 'logerror'})
df_corr.head()

NameError: name 'df' is not defined

In [15]:
from biokit.viz import corrplot
c = corrplot.Corrplot(df_corr)
# c.plot(colorbar=False, method='circle', shrink=.9, lower='circle')
c.plot()

ModuleNotFoundError: No module named 'biokit'

In [16]:
import prepare

ModuleNotFoundError: No module named 'prepare'

In [17]:
df = prepare.prep_zillow_data()
df.head()

NameError: name 'prepare' is not defined

In [18]:
df.shape

NameError: name 'df' is not defined

In [19]:
df.info()

NameError: name 'df' is not defined

### MVP prep is almost done.

At this point we realized that we had to take out the object columns in the dataframe in order to scale our numeric data. We took out the object columns in order for our min_max_scale function to run appropriately.

In [20]:
# Now on to splitting the data:

df.head()
df.shape

NameError: name 'df' is not defined

### For the MVP, we only need 3 columns, plus a target variable, so I'm making an MVP dataframe that isolates those columns:

In [21]:
mvp_df = df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']]
mvp_df.shape

NameError: name 'df' is not defined

In [22]:
# Importing the wrangle function:
import luke_wrangle

ModuleNotFoundError: No module named 'luke_wrangle'

In [23]:
import wrangle

wrangle.py functions loaded successfully


In [24]:
X_train, y_train, X_validate, y_validate, X_test, y_test = luke_wrangle.train_validate_test(mvp_df, 'taxvaluedollarcnt')

NameError: name 'luke_wrangle' is not defined

In [25]:
X_train.info()

NameError: name 'X_train' is not defined

In [26]:
X_train.shape, y_train.shape, X_validate.shape, X_test.shape

NameError: name 'X_train' is not defined

In [27]:
obj_cols = luke_wrangle.get_object_cols(df)
obj_cols

NameError: name 'luke_wrangle' is not defined

In [28]:
numeric_cols = luke_wrangle.get_numeric_X_cols(X_train, obj_cols)
numeric_cols

NameError: name 'luke_wrangle' is not defined

In [29]:
numeric_cols

NameError: name 'numeric_cols' is not defined

In [30]:
X_train.columns

NameError: name 'X_train' is not defined

In [31]:
X_train_scaled, X_validate_scaled, X_test_scaled = luke_wrangle.min_max_scale(X_train, X_validate, X_test, numeric_cols)

NameError: name 'luke_wrangle' is not defined

In [32]:
X_train_scaled.shape, X_validate_scaled.shape, X_test_scaled.shape

NameError: name 'X_train_scaled' is not defined

## Now we have an MVP dataset that has been split into X_train, etc. and Scaled

In [33]:
df.shape

NameError: name 'df' is not defined

In [34]:
X_train.shape, X_validate.shape, X_test.shape

NameError: name 'X_train' is not defined

In [35]:
numeric_cols_test = numeric_cols = luke_wrangle.get_numeric_X_cols(X_train, obj_cols)
len(numeric_cols_test)

NameError: name 'luke_wrangle' is not defined

In [36]:
X_train_scaled_full, X_validate_scaled_full, X_test_scaled_full = luke_wrangle.min_max_scale(X_train, X_validate, X_test, numeric_cols_test)

NameError: name 'luke_wrangle' is not defined

## Exploration

In [37]:
# Looking for variables that may have significance for our model...

In [38]:
df.head()

NameError: name 'df' is not defined

In [39]:
df.calculatedfinishedsquarefeet.mean()

NameError: name 'df' is not defined

In [40]:
alpha= .05


squareft = df[df.calculatedfinishedsquarefeet > 1800]
t, p = stats.ttest_1samp(squareft.calculatedfinishedsquarefeet, train.monthly_charges.mean())
print(f't = {t:.3f}')
print(f'p = {p:.3f}')

NameError: name 'df' is not defined

In [41]:
zillow_heatmap = df[['calculatedfinishedsquarefeet', 'bathroomcnt', 'bedroomcnt', 'yearbuilt', 'propertylandusetypeid', 'taxamount']]

corr = zillow_heatmap.corr()
corr

NameError: name 'df' is not defined

In [42]:
plt.rc('figure', figsize=(13, 10))
plt.rc('font', size=14)

In [43]:
cmap = sns.diverging_palette(15, 175, as_cmap = True)

sns.heatmap(corr, annot = True, cmap = cmap)
plt.show()

NameError: name 'corr' is not defined

In [44]:
sns.jointplot(y = "taxamount", x = "yearbuilt", data=df, kind='kde', height=5)
plt.show()

NameError: name 'df' is not defined

# Things to do
- Presentation and slides
- Readme
- Data Dictionary
- T-test
- Make visualizations using our 5 best features
- Answer these questions in explore:
    1. What independent variables are correlated with the dependent?
    2. Which independent variables are correlated with other independent variables?
- Markdown and Comment Notebooks
- Model.py
- Done - Acquire.py
- Done - Prep.py
- Done - Corr Test
- Done - Feature engineering

In [ ]:
# 




cmap = sns.diverging_palette(15, 175, as_cmap = True)

sns.heatmap(corr2, annot = True, cmap = cmap)
plt.show()